In [19]:
from IPython.display import Latex
# The Hamiltonian: 
# i,j: site index
# a,b: orbital index
# sigma: spin index
Latex(r"$H = -t\sum_{i\neq j,a,\sigma}{c_{ia\sigma}^\dagger c_{ja\sigma}} +  U \sum_i{n_{ia}n_{ia}} + U'\sum_{i,a<b}{(n_{ia\uparrow \
      }n_{ib\downarrow}+n_{ia\downarrow}n_{ib\uparrow})} + (U' - J)\sum_{i,a<b}{n_{ia\sigma}n_{ib\sigma}} - J\sum_{i,a<b}{\
      (d^\dagger_{ia\uparrow}d_{ia\downarrow}d^\dagger_{ib\uparrow}d_{ib\downarrow} + d^\dagger_{ia\uparrow}d_{ia\downarrow}\
      d^\dagger_{ib\downarrow}d_{ib\uparrow} + d^\dagger_{ia\uparrow}d^\dagger_{ia\downarrow}\
      d_{ib\uparrow}d_{ib\downarrow} + h.c.)}$")

<IPython.core.display.Latex object>

In [20]:
# ED of 1D Hubbard model with 2M band (M does not contains spin index)

import numpy as np
from matplotlib import pyplot as plt
import scipy.sparse as spr
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import eigs, eigsh

# N: number of sites
# M: number of bands at single site
# kesi: prepare for antisymmetrization
N = 2
M = 2
kesi = -1.0
dim = 2**(2*M*N)


del dim
t = 1.0
U = 0.0
J = 0.00
U_2 = U - 2*J
# if U = 0,then J and U_2 (U') should be 0
for num in range(0,2*N*M+1):
    
# subspace basis, first get all basis whose particle number are num
    basis = []
    dim = 0
    for cn in range(0,2**(2*M*N)):
        if bin(cn)[2:].count('1') == num:
            dim = dim + 1
            basis.append(cn)
    print('dimension of Hamultionian: '+str(dim))
    H = np.zeros((dim,dim))
    N_occsub = np.identity((dim))*num
    T = np.zeros((dim,dim))
    V = np.zeros((dim,dim))
    V1 = np.zeros((dim,dim))
    V2 = np.zeros((dim,dim))
    V3 = np.zeros((dim,dim))

# construct the  Hamiltonian (1D)
    for i in range(0,N):
        for j in range(0,2*M):
            for cn in range(0,dim):
                length = len(bin(basis[cn]))-2
                str_cn = bin(basis[cn])[2:]
                
                # complete binary representation, for example, from  10 to 0010
                for k in range(0,2*M*N-length):
                    str_cn = '0' + str_cn 
                # kinetic energy
                # odd: up ; even: down
                if i > 0 and i < N-1: 
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j) + 2*M)] == '1':
                        # 1-> 0
                        str_cntmp1[2*M*N-1-((2*M*i+j) + 2*M)] = '0'
                        i_orb = 2*M*N-1-((2*M*i+j) + 2*M)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power

                        # 0-> 1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j) - 2*M)] == '1':
                        # 1-> 0
                        str_cntmp1[2*M*N-1-((2*M*i+j) - 2*M)] = '0'
                        i_orb = 2*M*N-1-((2*M*i+j) - 2*M)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power
                        
                        # 0-> 1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]

                if i == 0:
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j) + 2*M)] == '1':
                        # 1-> 0
                        str_cntmp1[2*M*N-1-((2*M*i+j) + 2*M)] = '0'
                        i_orb = 2*M*N-1-((2*M*i+j) + 2*M)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power
                        
                        # 0-> 1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-(2*M*(N - 1) + (2*M*i+j))] == '1':
                        #1->0
                        str_cntmp1[2*M*N-1-(2*M*(N - 1) + (2*M*i+j))] = '0'
                        i_orb = 2*M*N-1-(2*M*(N - 1) + (2*M*i+j))
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power

                        # 0->1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]

                if i == N - 1 :
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j) - 2*M)] == '1':
                        # 1->0
                        str_cntmp1[2*M*N-1-((2*M*i+j) - 2*M)] = '0'
                        i_orb = 2*M*N-1-((2*M*i+j) - 2*M)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power
                        
                        # 0->1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]
                    str_cntmp1 = list(str_cn)
                    if str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-j] == '1':
                        # 1->0
                        str_cntmp1[2*M*N-1-j] = '0'
                        i_orb = 2*M*N-1-j
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign2 = (kesi)**inv_power
                        
                        # 0->1
                        str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                        i_orb = 2*M*N-1-(2*M*i+j)
                        str_cntmp2 = ''.join(str_cntmp1)
                        inv_power = str_cntmp2[i_orb + 1:].count('1')
                        fsign1 = (kesi)**inv_power
                        
                        str_newcn1 = ''.join(str_cntmp1)
                        str_newcn1 = '0b' + str_newcn1
                        newcn = basis.index(int(str_newcn1,2))
                        T[newcn,cn] =  1.0 * fsign2*fsign1 + T[newcn,cn]
            
            # four terms fermi operators, on-site term
                if M == 1 :
                    if j%2 == 0:
                        str_cntmp1 = list(str_cn)
                        if str_cntmp1[2*M*N-1-(2*M*i+j)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j) + 1)] == '1':
                            V[cn,cn] = 1.0 + V[cn,cn] 

                if M > 1:
                    if j%2 == 0:
                        str_cntmp1 = list(str_cn)
                        if str_cntmp1[2*M*N-1-(2*M*i+j)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j) + 1)] == '1':
                            V[cn,cn] = 1.0 + V[cn,cn]

                        for k in range(0,j,2):
                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j) + 1)] == '1':
                                V1[cn,cn] = 1.0 + V1[cn,cn]
                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k+1)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j))] == '1':
                                V1[cn,cn] = 1.0 + V1[cn,cn]

                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j))] == '1':
                                V2[cn,cn] = 1.0 + V1[cn,cn]
                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k+1)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j+1))] == '1':
                                V2[cn,cn] = 1.0 + V2[cn,cn]

                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+k+1))] == '1' and str_cntmp1[2*M*N-1-(2*M*i+j)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j+1))] == '1' :
                                
                                str_cntmp1[2*M*N-1-((2*M*i+j+1))] = '0'
                                i_orb = 2*M*N-1-((2*M*i+j+1))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign4 = (kesi)**inv_power
                                 
                                str_cntmp1[2*M*N-1-(2*M*i+j)] = '1'
                                i_orb = 2*M*N-1-(2*M*i+j)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign3 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-((2*M*i+k+1))] = '0'
                                i_orb = 2*M*N-1-((2*M*i+k+1))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign2 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-(2*M*i+k)] = '1'
                                i_orb = 2*M*N-1-(2*M*i+k)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign1 = (kesi)**inv_power
                                
                                str_newcn1 = ''.join(str_cntmp1)
                                str_newcn1 = '0b' + str_newcn1
                                newcn = basis.index(int(str_newcn1,2))
                                V3[newcn,cn] =  1.0 * fsign4*fsign3*fsign2*fsign1 + V3[newcn,cn]
                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+k+1))] == '1' and str_cntmp1[2*M*N-1-(2*M*i+j+1)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+j))] == '1' :
                                
                                str_cntmp1[2*M*N-1-((2*M*i+j))] = '0'
                                i_orb = 2*M*N-1-((2*M*i+j))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign4 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-(2*M*i+j+1)] = '1'
                                i_orb = 2*M*N-1-(2*M*i+j+1)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign3 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-((2*M*i+k+1))] = '0'
                                i_orb = 2*M*N-1-((2*M*i+k+1))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign2 = (kesi)**inv_power
                                  
                                str_cntmp1[2*M*N-1-(2*M*i+k)] = '1'
                                i_orb = 2*M*N-1-(2*M*i+k)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign1 = (kesi)**inv_power
                                
                                str_newcn1 = ''.join(str_cntmp1)
                                str_newcn1 = '0b' + str_newcn1
                                newcn = basis.index(int(str_newcn1,2))
                                V3[newcn,cn] =  1.0 * fsign4*fsign3*fsign2*fsign1 + V3[newcn,cn]
                            str_cntmp1 = list(str_cn)
                            if str_cntmp1[2*M*N-1-(2*M*i+k)] == '0' and str_cntmp1[2*M*N-1-((2*M*i+k+1))] == '0' and str_cntmp1[2*M*N-1-(2*M*i+j)] == '1' and str_cntmp1[2*M*N-1-((2*M*i+j+1))] == '1' : 

                                str_cntmp1[2*M*N-1-((2*M*i+j+1))] = '0'
                                i_orb = 2*M*N-1-((2*M*i+j+1))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign4 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-(2*M*i+j)] = '0'
                                i_orb = 2*M*N-1-(2*M*i+j)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign3 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-((2*M*i+k+1))] = '1'
                                i_orb = 2*M*N-1-((2*M*i+k+1))
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign2 = (kesi)**inv_power
                                
                                str_cntmp1[2*M*N-1-(2*M*i+k)] = '1'
                                i_orb = 2*M*N-1-(2*M*i+k)
                                str_cntmp2 = ''.join(str_cntmp1)
                                inv_power = str_cntmp2[i_orb + 1:].count('1')
                                fsign1 = (kesi)**inv_power
                                
                                str_newcn1 = ''.join(str_cntmp1)
                                str_newcn1 = '0b' + str_newcn1
                                newcn = basis.index(int(str_newcn1,2)) 
                                V3[newcn,cn] =  1.0 * fsign4*fsign3*fsign2*fsign1 + V3[newcn,cn]
                        

# diagonalization
    H = -t*T + U*V + U_2*V1 + (U_2 - J)*V2 - J*V3 - J*np.conjugate(np.transpose(V3))
    eig_val,eig_vec = np.linalg.eigh(H)
    print('eigenvalues: ')
    print(eig_val)
    print(' ')


dimension of Hamultionian: 1
eigenvalues: 
[0.]
 
dimension of Hamultionian: 8
eigenvalues: 
[-2. -2. -2. -2.  2.  2.  2.  2.]
 
dimension of Hamultionian: 28
eigenvalues: 
[-3.61995025e+00 -3.59751476e+00 -3.57523291e+00 -3.55310439e+00
 -3.55310439e+00 -3.50930601e+00 -6.39400389e-16 -1.64621193e-16
 -1.37353043e-16 -1.01623462e-16 -6.08242428e-18 -1.21568873e-32
  3.10266583e-17  1.01630771e-16  2.30009447e-16  5.84446369e-16
  8.00000000e-01  8.50000000e-01  9.00000000e-01  9.50000000e-01
  9.50000000e-01  1.05000000e+00  4.41995025e+00  4.44751476e+00
  4.47523291e+00  4.50310439e+00  4.50310439e+00  4.55930601e+00]
 
dimension of Hamultionian: 56
eigenvalues: 
[-4.69581475 -4.69581475 -4.55557325 -4.55557325 -1.18400982 -1.18400982
 -1.18394709 -1.18394709 -1.13039739 -1.13039739 -1.13031901 -1.13031901
 -1.08075137 -1.08075137 -1.08068406 -1.08068406 -1.05       -1.05
 -1.05       -1.05       -0.97375967 -0.97375967 -0.97358877 -0.97358877
  0.20555175  0.20555175  0.73367432  0